In [ ]:
from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder \
    .appName("SimpleApp") \
    .getOrCreate()



In [15]:

# Read CSV File
df_fare = spark.read.option("header", True).csv("data_lake/trip_fare_1.csv")
df_trip = spark.read.option("header", True).csv("data_lake/trip_data_1.csv")

In [19]:
df_trip.show()

+--------------------+--------------------+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
|           medallion|        hack_license|vendor_id|rate_code|store_and_fwd_flag|    pickup_datetime|   dropoff_datetime|passenger_count|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|
+--------------------+--------------------+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
|89D227B655E5C82AE...|BA96DE419E711691B...|      CMT|        1|                 N|2013-01-01 15:11:48|2013-01-01 15:18:10|              4|              382|         1.00|      -73.978165|      40.757977|       -73.989838|       40.751171|
|0BD7C8F5BA12B88E0...|9FD8F69F0804BDB55...| 

In [17]:
df_fare.show()

+--------------------+--------------------+----------+-------------------+-------------+------------+----------+--------+-----------+-------------+-------------+
|           medallion|        hack_license| vendor_id|    pickup_datetime| payment_type| fare_amount| surcharge| mta_tax| tip_amount| tolls_amount| total_amount|
+--------------------+--------------------+----------+-------------------+-------------+------------+----------+--------+-----------+-------------+-------------+
|89D227B655E5C82AE...|BA96DE419E711691B...|       CMT|2013-01-01 15:11:48|          CSH|         6.5|         0|     0.5|          0|            0|            7|
|0BD7C8F5BA12B88E0...|9FD8F69F0804BDB55...|       CMT|2013-01-06 00:18:35|          CSH|           6|       0.5|     0.5|          0|            0|            7|
|0BD7C8F5BA12B88E0...|9FD8F69F0804BDB55...|       CMT|2013-01-05 18:49:41|          CSH|         5.5|         1|     0.5|          0|            0|            7|
|DFD2202EE08F7A8DC...|51EE87

In [18]:
df_trip.show()

+--------------------+--------------------+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
|           medallion|        hack_license|vendor_id|rate_code|store_and_fwd_flag|    pickup_datetime|   dropoff_datetime|passenger_count|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|
+--------------------+--------------------+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
|89D227B655E5C82AE...|BA96DE419E711691B...|      CMT|        1|                 N|2013-01-01 15:11:48|2013-01-01 15:18:10|              4|              382|         1.00|      -73.978165|      40.757977|       -73.989838|       40.751171|
|0BD7C8F5BA12B88E0...|9FD8F69F0804BDB55...| 